In [3]:
import cupy as cp
from numba import cuda

In [4]:
import cupy as cp

kernel_code = r'''
// Double precision atomicAdd implementation
__device__ double atomicAdd_double(double* address, double val)
{
    unsigned long long int* address_as_ull = (unsigned long long int*)address;
    unsigned long long int old = *address_as_ull;
    unsigned long long int assumed;
    do {
        assumed = old;
        old = atomicCAS(address_as_ull, assumed,
                       __double_as_longlong(val + __longlong_as_double(assumed)));
    } while (assumed != old);
    return __longlong_as_double(old);
}

extern "C" __global__ void process_combinations(
    const double* Mmat,
    const double* poly,
    const int* idx,
    const double* u_field,double* umat,
    int order,
    int N,
    int batch_size
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int total_combinations = order * order;
    
    if (tid < total_combinations) {
        int i = tid / order;
        int j = tid % order;
        
        // Process one i,j combination per thread
        for (int b = 0; b < batch_size; b++) {
            double temparr = 0.0;
            
            // Manual einsum computation
            for (int p = 0; p < 4; p++) {
                for (int q = 0; q < 4; q++) {
                    temparr += Mmat[i * 4 + p] * Mmat[j * 4 + q] * 
                              poly[b * 8 + p] * poly[b * 8 + 4 + q];
                }
            }
            
            // Compute indices
            int slx = (idx[b * 2] - 1 + i) % N;
            int sly = (idx[b * 2 + 1] - 1 + j) % N;
            
            // Update umat using custom atomicAdd for double precision
            atomicAdd_double(&umat[b * 2], u_field[slx * N * 2 + sly * 2] * temparr);
            atomicAdd_double(&umat[b * 2 + 1], u_field[slx * N * 2 + sly * 2 + 1] * temparr);
        }
    }
}
'''

def parallel_computation_cuda(order, Mmat, poly, idx, N, u_field, threads_per_block=256):
    # Ensure all inputs are float64
    Mmat_gpu = cp.asarray(Mmat, dtype=cp.float64)
    poly_gpu = cp.asarray(poly, dtype=cp.float64)
    idx_gpu = cp.asarray(idx, dtype=cp.int32)
    u_field_gpu = cp.asarray(u_field, dtype=cp.float64)
    
    batch_size = poly_gpu.shape[0]
    umat_gpu = cp.zeros((batch_size, 2), dtype=cp.float64)
    
    # Compile the kernel
    module = cp.RawModule(code=kernel_code)
    kernel = module.get_function('process_combinations')
    
    # Calculate grid dimensions
    total_threads = order * order
    blocks = (total_threads + threads_per_block - 1) // threads_per_block
    
    # Launch kernel
    kernel((blocks,), (threads_per_block,),
          (Mmat_gpu, poly_gpu, idx_gpu, u_field_gpu, umat_gpu,
           cp.int32(order), cp.int32(N), cp.int32(batch_size)))
    
    return umat_gpu

# Example usage:
# result = parallel_computation_cuda(order, Mmat, poly, idx, N, u_field)

In [29]:
order = 200
N = 1024
Np = N//3
Mat = cp.random.random((order, order)).astype(cp.float64)
poly = cp.random.random((Np,2,order),dtype=cp.float64)
idx = cp.random.randint(0, N, (Np,2), dtype=int)
u_field = cp.random.random((N, N, 2)).astype(cp.float64)*10




In [34]:
%%time
d1 = parallel_computation_cuda(order, Mat, poly, idx, N, u_field)

CPU times: user 655 µs, sys: 162 µs, total: 817 µs
Wall time: 828 µs


In [26]:
def normal_func(order, Mmat, poly, idx, N, u_field):
    umat = idx*0.0
    for i in range(order):
        for j in range(order):
            temparr = cp.einsum('p,q,...p,...q->...',Mmat[i],Mmat[j],poly[...,0,:],poly[...,1,:]) #! For saving computations
            slx = (idx[:,0]-1 + i)%N #! For saving computations
            sly= (idx[:,1]-1 + j)%N #! For saving computations
            
            umat  += u_field[slx,sly,...]*temparr[:,None]
    return umat

In [28]:
%%time
d2 = normal_func(order, Mat, poly, idx, N, u_field)

CPU times: user 1.5 s, sys: 343 ms, total: 1.85 s
Wall time: 1.85 s


In [1]:
import concurrent.futures
import itertools

# Approach 1: Using concurrent.futures ThreadPoolExecutor
def process_ij_combination(args):
    i, j, Mmat_gpu, poly_gpu, idx_gpu, N, u_field_gpu = args
    batch_size = poly_gpu.shape[0]
    
    # Compute temparr using einsum
    temparr = cp.einsum('p,q,np,nq->n',
                       Mmat_gpu[i],
                       Mmat_gpu[j],
                       poly_gpu[...,0,:],
                       poly_gpu[...,1,:])
    
    # Compute indices
    slx = (idx_gpu[:,0] - 1 + i) % N
    sly = (idx_gpu[:,1] - 1 + j) % N
    
    # Get u_field values and multiply with temparr
    return u_field_gpu[slx, sly] * temparr[:,None]

def parallel_computation_threads(order, Mmat, poly, idx, N, u_field):
    # Move data to GPU once
    Mmat_gpu = cp.asarray(Mmat)
    poly_gpu = cp.asarray(poly)
    idx_gpu = cp.asarray(idx)
    u_field_gpu = cp.asarray(u_field)
    
    # Initialize output array
    batch_size = poly_gpu.shape[0]
    umat_gpu = cp.zeros((batch_size, 2), dtype=poly_gpu.dtype)
    
    # Create all combinations of i,j
    ij_combinations = [(i, j, Mmat_gpu, poly_gpu, idx_gpu, N, u_field_gpu) 
                      for i, j in itertools.product(range(order), range(order))]
    
    # Process combinations in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_ij_combination, ij_combinations))
    
    # Sum all results
    for result in results:
        umat_gpu += result
    
    return cp.asnumpy(umat_gpu)


In [ ]:
p